<a href="https://colab.research.google.com/github/EderLara/IA-Innovador-Talento-Tech/blob/main/Ejemplo_de_Redes_generativas_adversariales_(GAN)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Qué son las Redes Generativas Adversariales (GANs)?

Imagina que tienes dos entidades: un Falsificador (el Generador) y un Detective (el Discriminador).

1. El Generador (G): Su trabajo es crear "falsificaciones" (datos sintéticos) que se parezcan lo más posible a los datos reales. Empieza creando cosas al azar, pero con el tiempo aprende a hacer falsificaciones cada vez mejores. Por ejemplo, si queremos generar imágenes de gatos, el Generador intentará crear imágenes que parezcan gatos reales. Su entrada es un vector de ruido aleatorio (llamado espacio latente) y su salida es un dato sintético (una imagen, en nuestro ejemplo).
2. El Discriminador (D): Su trabajo es diferenciar entre los datos reales y las "falsificaciones" creadas por el Generador. Recibe un dato (una imagen) y debe decidir si es real (proveniente del conjunto de datos original) o falso (creado por G). Su salida es una probabilidad (por ejemplo, 0 para falso, 1 para real).

## El Juego Adversarial (El Entrenamiento):

Ambos, Generador y Discriminador, son redes neuronales que aprenden y mejoran compitiendo entre sí:

* El Discriminador se entrena mostrándole ejemplos reales y ejemplos falsos generados por G. Aprende a identificar las características que distinguen lo real de lo falso. Su objetivo es volverse muy bueno detectando las falsificaciones (maximizar su precisión).
* El Generador se entrena basándose en qué tan bien logra engañar al Discriminador. Si el Discriminador detecta fácilmente sus falsificaciones, el Generador ajusta sus parámetros para crear datos sintéticos más convincentes. Su objetivo es que el Discriminador clasifique sus creaciones como reales (minimizar la capacidad del Discriminador para detectar sus falsificaciones).

Este proceso se repite muchas veces. A medida que el Discriminador mejora, el Generador se ve forzado a mejorar también para poder seguir engañándolo. A su vez, un Generador mejor obliga al Discriminador a volverse aún más astuto. Eventualmente, si todo va bien, el Generador aprende a crear datos sintéticos muy realistas, casi indistinguibles de los reales.

# Ejemplo Práctico Paso a Paso: Generar Dígitos Manuscritos (como los del dataset MNIST)

Vamos a detallar conceptualmente los pasos para entrenar una GAN que genere imágenes de dígitos escritos a mano (0 al 9), similares a los del famoso dataset MNIST.

**Objetivo:** Crear una GAN capaz de generar imágenes nuevas y realistas de 28x28 píxeles que parezcan dígitos escritos a mano.

**Componentes:**

1. Dataset Real: Usaremos el dataset MNIST, que contiene miles de imágenes reales de 28x28 píxeles de dígitos manuscritos (0-9). Estos son los ejemplos "reales".
2. Generador (G):
  * Entrada: Un vector de ruido aleatorio (por ejemplo, de 100 dimensiones). Este ruido es la "semilla" creativa.
  * Arquitectura (Conceptual): Una red neuronal (por ejemplo, con capas densas o capas convolucionales transpuestas si es una DCGAN) que transforma el vector de ruido de 100 dimensiones en una imagen de 28x28 píxeles. Podría usar funciones de activación como ReLU o LeakyReLU en capas intermedias y una función Tanh o Sigmoid en la capa final para escalar los píxeles al rango adecuado (e.g., [-1, 1] o [0, 1]).
  * Salida: Una imagen sintética de 28x28 píxeles.

3. Discriminador (D):
  * Entrada: Una imagen de 28x28 píxeles (puede ser real del dataset MNIST o falsa generada por G).
  * Arquitectura (Conceptual): Una red neuronal convolucional (CNN) típica para clasificación de imágenes. Tendrá capas convolucionales (para extraer características), quizás capas de pooling, capas densas y funciones de activación (comúnmente LeakyReLU para evitar gradientes nulos).
  * Salida: Un único número (una probabilidad entre 0 y 1) que indica la probabilidad de que la imagen de entrada sea "real". Se usa una función Sigmoid en la capa final.

## **Proceso de Entrenamiento (Iterativo):**
El entrenamiento se realiza por épocas (pasadas completas por el dataset) y en cada época, por lotes (batches) de datos. Para cada lote:

**Paso 1: Preparar Datos**

  * Toma un lote de imágenes reales del dataset MNIST (ej. 64 imágenes).
  * Genera un lote de imágenes falsas:
    * Crea un lote de vectores de ruido aleatorio (ej. 64 vectores de 100 dimensiones).
    * Pasa estos vectores a través del Generador (G) para obtener 64 imágenes falsas.

**Paso 2: Entrenar el Discriminador (D)**

  * Objetivo: Que D aprenda a distinguir las imágenes reales de las falsas de este lote.
  * Alimenta al Discriminador con las imágenes reales. Las etiquetas objetivo para estas imágenes son 1 (o "real"). Calcula la pérdida del Discriminador para estas imágenes (qué tan lejos está su predicción de 1).
  * Alimenta al Discriminador con las imágenes falsas (generadas en el Paso 1). Las etiquetas objetivo para estas imágenes son 0 (o "falso"). Calcula la pérdida del Discriminador para estas imágenes (qué tan lejos está su predicción de 0).
  * Suma las dos pérdidas (real y falsa).
  * Calcula los gradientes de esta pérdida total con respecto a los pesos del Discriminador.
  * Actualiza los pesos del Discriminador usando un optimizador (como Adam) para minimizar esta pérdida combinada. Importante: En este paso, los pesos del Generador NO se actualizan.

**Paso 3: Entrenar el Generador (G)**

  * Objetivo: Que G aprenda a generar imágenes que engañen al Discriminador (que D las clasifique como 1).
  * Genera un nuevo lote de imágenes falsas (usando nuevos vectores de ruido y el Generador G actual).
  * Pasa estas imágenes falsas a través del Discriminador (D). Importante: Ahora, para entrenar a G, pretendemos que estas imágenes falsas son "reales". Por lo tanto, las etiquetas objetivo que usamos para calcular la pérdida del Generador son 1.
  * Calcula la pérdida del Generador: qué tan lejos está la predicción del Discriminador (para las imágenes falsas) de la etiqueta objetivo 1.
  * Calcula los gradientes de esta pérdida con respecto a los pesos del Generador. **Importante:** *En este paso, los pesos del Discriminador se mantienen fijos (no se actualizan)*. Solo queremos saber cómo debe cambiar el Generador para engañar mejor al Discriminador actual.
  * Actualiza los pesos del Generador usando un optimizador para minimizar su pérdida (es decir, para maximizar la probabilidad de que D clasifique sus imágenes como reales).

**Paso 4: Repetir**

  * Repite los Pasos 1 a 3 para el siguiente lote de datos.
  * Continúa este proceso durante muchas épocas.

**Resultado Esperado:**

  * Al principio, el Generador producirá ruido sin sentido. El Discriminador aprenderá rápidamente a distinguirlo de los dígitos reales.
  * A medida que el Generador recibe retroalimentación (a través de los gradientes del Discriminador), empezará a producir formas más coherentes que se asemejan vagamente a dígitos.
  * El Discriminador se volverá más exigente, forzando al Generador a refinar aún más sus salidas.
  * Después de suficiente entrenamiento, el Generador debería ser capaz de producir imágenes de dígitos manuscritos que un humano (¡y el Discriminador!) tendría dificultades para distinguir de los reales. Podrás darle un vector de ruido aleatorio y obtener una imagen nueva y única de un dígito.

**Consideraciones Clave:**

  * Equilibrio: Es crucial mantener un equilibrio en el entrenamiento. Si el Discriminador se vuelve demasiado bueno muy rápido, el Generador no podrá aprender. Si el Generador engaña fácilmente al Discriminador desde el principio, no recibirá gradientes útiles para mejorar.
  * Funciones de Pérdida: Comúnmente se usa la Entropía Cruzada Binaria (Binary Cross-Entropy) tanto para el Discriminador como para el Generador.
  * Hiperparámetros: La tasa de aprendizaje, el tamaño del lote, la arquitectura de las redes y el tamaño del vector de ruido son hiperparámetros importantes que necesitan ser ajustados.
  * Colapso de Modos (Mode Collapse): A veces, el Generador puede aprender a producir solo un tipo o un conjunto muy limitado de salidas que engañan bien al Discriminador, en lugar de aprender toda la diversidad del dataset real. Es un desafío común en el entrenamiento de GANs.

Este ejemplo pretende dar una visión general del flujo de trabajo y la lógica detrás del entrenamiento de una GAN.

La implementación real implicaría usar librerías como TensorFlow o PyTorch para definir las redes neuronales y el bucle de entrenamiento.

*El siguiente código está diseñado para ser lo más claro posible y puedes ejecutarlo en entornos como Google Colab (que te da acceso gratuito a GPUs, lo cual es muy recomendable para entrenar GANs).*

In [ ]:
"""
DCGAN Simple en Keras/TensorFlow para generar dígitos MNIST.
"""

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os # Para guardar imágenes

# --- 1. Hiperparámetros y Configuración ---
BUFFER_SIZE = 60000                         # Tamaño del buffer para barajar el dataset (igual al tamaño de MNIST)
BATCH_SIZE = 256                            # Tamaño del lote
EPOCHS = 50                                 # Número de épocas de entrenamiento (puede necesitar más para mejores resultados)
latent_dim = 100                            # Dimensión del vector de ruido de entrada para el generador
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# Optimizers - Adam es común para GANs
# Usar beta_1=0.5 a veces ayuda a estabilizar el entrenamiento
generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

# Función de Pérdida
# Usamos BinaryCrossentropy porque el discriminador da una probabilidad (0 o 1)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False) # False porque la última capa del D tiene Sigmoid

# Directorio para guardar imágenes generadas
if not os.path.exists('gan_images_mnist'):
    os.makedirs('gan_images_mnist')

In [ ]:
# --- 2. Cargar y Preparar el Dataset MNIST ---
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Añadir dimensión de canal y normalizar las imágenes al rango [-1, 1]
# (tanh en la salida del generador produce valores en [-1, 1])
train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, channels).astype('float32')
train_images = (train_images - 127.5) / 127.5

# Crear lotes y barajar el dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# --- 3. Construir el Generador (G) ---
# Transforma un vector de ruido (latent_dim) en una imagen (28x28x1)
def make_generator_model():
    model = tf.keras.Sequential()
    # Capa densa inicial y reshape para empezar a formar la imagen
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # None es el tamaño del batch

    # Capas convolucionales transpuestas (Deconvolution) para aumentar tamaño
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Capa final con activación tanh para que la salida esté en [-1, 1]
    model.add(layers.Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, img_rows, img_cols, channels)

    return model

In [ ]:
# --- 4. Construir el Discriminador (D) ---
# Clasifica una imagen (28x28x1) como real (1) o falsa (0)
def make_discriminator_model():
    model = tf.keras.Sequential()
    # Capas convolucionales para extraer características
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3)) # Dropout para regularización

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Aplanar y capa densa final con Sigmoid para probabilidad
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid')) # Salida de probabilidad

    return model

# Crear instancias de los modelos
generator = make_generator_model()
discriminator = make_discriminator_model()

print("--- Resumen del Generador ---")
generator.summary()
print("\n--- Resumen del Discriminador ---")
discriminator.summary()

In [ ]:
# --- 5. Definir las Funciones de Pérdida ---

# Pérdida del Discriminador: Compara predicciones en imágenes reales con un array de 1s,
# y predicciones en imágenes falsas con un array de 0s.
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output) # Queremos que prediga 1 para reales
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output) # Queremos que prediga 0 para falsas
    total_loss = real_loss + fake_loss
    return total_loss

# Pérdida del Generador: Intenta engañar al discriminador.
# Compara las predicciones del discriminador sobre imágenes falsas con un array de 1s.
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output) # Queremos que D prediga 1 para las falsas

In [ ]:
# --- 6. Definir el Paso de Entrenamiento ---
# Usamos tf.function para compilar la función en un grafo, lo que acelera el entrenamiento.
@tf.function
def train_step(images):
    # 1. Generar ruido y luego imágenes falsas
    noise = tf.random.normal([BATCH_SIZE, latent_dim])

    # Usamos GradientTape para registrar las operaciones para la diferenciación automática
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        # 2. Obtener predicciones del Discriminador
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # 3. Calcular pérdidas
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # 4. Calcular gradientes
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # 5. Aplicar gradientes para actualizar pesos
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss # Devolver pérdidas para monitoreo

In [ ]:
# --- 7. Helper para Generar y Guardar Imágenes ---
# Usaremos un conjunto fijo de vectores de ruido para ver cómo evoluciona la generación
seed = tf.random.normal([16, latent_dim]) # Generaremos 16 imágenes de ejemplo

def generate_and_save_images(model, epoch, test_input):
    # `training=False` para que capas como BatchNormalization funcionen en modo inferencia.
    predictions = model(test_input, training=False)
    # Re-escalar de [-1, 1] a [0, 1] para mostrar/guardar
    predictions = (predictions + 1) / 2.0

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Si es a color, usar predictions[i, :, :, :]
        plt.imshow(predictions[i, :, :, 0] * 255.0, cmap='gray') # Multiplicar por 255 si se guarda como entero
        plt.axis('off')

    plt.savefig('gan_images_mnist/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show() # También mostrar en el notebook/consola si es posible

In [ ]:
# --- 8. Bucle de Entrenamiento ---
def train(dataset, epochs):
    print("\n--- Iniciando Entrenamiento ---")
    start_time = time.time()

    for epoch in range(epochs):
        epoch_start_time = time.time()
        gen_loss_epoch = []
        disc_loss_epoch = []

        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            gen_loss_epoch.append(g_loss)
            disc_loss_epoch.append(d_loss)

        # Calcular pérdidas promedio por época
        avg_gen_loss = np.mean(gen_loss_epoch)
        avg_disc_loss = np.mean(disc_loss_epoch)

        # Producir imágenes al final de la época (o cada N épocas)
        generate_and_save_images(generator, epoch + 1, seed)

        # Imprimir progreso
        print(f'Época {epoch + 1}/{epochs} completada en {time.time() - epoch_start_time:.2f} seg')
        print(f'  Pérdida del Generador: {avg_gen_loss:.4f}, Pérdida del Discriminador: {avg_disc_loss:.4f}')

    # Generar imágenes después del entrenamiento final
    generate_and_save_images(generator, epochs, seed)
    print(f'\nEntrenamiento completado en {(time.time() - start_time)/60:.2f} minutos.')              # Utilizar el

In [ ]:
# --- Iniciar el entrenamiento ---
train(train_dataset, EPOCHS)

# --- Opcional: Guardar los modelos ---
# generator.save('generator_mnist_dcgan.h5')
# discriminator.save('discriminator_mnist_dcgan.h5')